<a href="https://www.kaggle.com/code/lonnieqin/chatbot-with-mixtral-8x7b?scriptVersionId=164159100" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Chatbot with Mixtral 8x7B

This Jupyter notebook demonstrates how to build a chatbot using Mixtral 8x7B, a large language model from [Mixtral AI](https://mixtral.ai). Mixtral is a powerful and fast model adaptable to many use-cases. While being 6x faster, it matches or outperform Llama 2 70B on all benchmarks, speaks many languages, has natural coding abilities. It handles 32k sequence length. Mixtral 8x7B even have comparable performance to GPT-3.5.  The notebook covers the following steps:

1. Loading the Mixtral 8x7B model into a Kaggle notebook
2. Creating a simple chatbot that responds to user input or pre-defined prompts

The notebook will also includes a discussion of the limitations of chatbots and how they can be used to improve customer service and provide other valuable services.

## References
* https://github.com/dvmazur/mixtral-offloading

## Configuration

In [1]:
class CFG:
    
    ## Whether to interact with the chatbot interactively
    is_interactive = False

    prompts = [
        "Could you tell me something about Elon Musk?",
        "Could you build an IMDB text classifier using Tensorflow?",
        "Could you build an IMDB text classifier using Pytorch?"
    ]

## Install and import libraries

In [2]:
!git clone https://github.com/dvmazur/mixtral-offloading
!cd mixtral-offloading && pip install -q -r requirements.txt
!pip install triton
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

Cloning into 'mixtral-offloading'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 290 (delta 54), reused 45 (delta 37), pack-reused 214
Receiving objects: 100% (290/290), 264.75 KiB | 8.82 MiB/s, done.
Resolving deltas: 100% (165/165), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 9.3 MB/s eta 0:00:00
/kaggle/working/Mixtral-8x7B-Instruct-v0.1-offloading-demo


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import sys
import time
sys.path.append("mixtral-offloading")
import torch
from IPython.display import display, Markdown
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers import TextStreamer
from transformers.utils import logging as hf_logging
from src.build_model import OffloadConfig, QuantConfig, build_model

hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

## Initialize model

In [4]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
offload_per_layer = 4
# offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Loading experts:   0%|          | 0/32 [00:00<?, ?it/s]

## Run the model

In [5]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None
seq_len = 0
i = 0
while True:
    
    if CFG.is_interactive:
        print("User: ", end="")
        user_input = input()
    else:
        print("User: ", end="")
        if i >= len(CFG.prompts):
            break
        user_input = CFG.prompts[i]
        i += 1
    print(f"{user_input}\n")

    user_entry = dict(role="user", content=user_input)
    input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(device)

    if past_key_values is None:
        attention_mask = torch.ones_like(input_ids)
    else:
        seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
        attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=device)
    print("Mixtral: ", end="")
    begin = time.time()
    result = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        past_key_values=past_key_values,
        streamer=streamer,
        do_sample=True,
        temperature=0.9,
        top_p=0.9,
        max_new_tokens=1024,
        pad_token_id=tokenizer.eos_token_id,
        return_dict_in_generate=True,
        output_hidden_states=True,
    )
    elapsed = time.time() - begin
    print(f"Elapsed time: {elapsed:.2f}s")
    print("\n")
    sequence = result["sequences"]
    past_key_values = result["past_key_values"]
    

User: Could you tell me something about Elon Musk?

Mixtral: 

2024-02-24 19:22:37.176461: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-24 19:22:37.176577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-24 19:22:37.356135: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Elon Musk is a South African-born Canadian-American entrepreneur and business magnate who is the founder, CEO, and CTO of SpaceX, CEO of Neuralink, and co-founder, CEO, and product architect of Tesla Inc. He is an inventor, engineer, industrial designer, and philanthropist and is known for his work in the fields of green energy, artificial intelligence, and space exploration.

Musk was born on June 27, 1971, in Pretoria, South Africa. He attended the University of Pretoria, where he completed his bachelor's degree in business administration and economics in just two years. After completing his undergraduate degree, Musk moved to the United States and pursued a PhD in energy physics at Stanford University, but he dropped out of the program after only two days to start his own company, which would eventually become PayPal.

In 2002, Musk founded Space Exploration Technologies (SpaceX) with the goal of reducing space transportation costs and enabling the colonization of Mars. SpaceX has s

## Conclusion

It is cool to run a Mixtral 8x7B in Kaggle Notebook. This model can generate some high quality contents even with 4 bit quantization. However the speed is slow, it still takes 500 seconds to generate an answer with 1024 content length. This is because the model is very large and requires a lot of resources to run. However, the results are still very impressive.

Here are some specific examples of how Mixtral 8x7B can be used to improve customer service and provide other valuable services:

* **Mixtral 8x7B can be used to create a knowledge base of frequently asked questions (FAQs).** This can help your customers find answers to their questions quickly and easily, without having to contact customer service.
* **Mixtral 8x7B can be used to provide live chat support.** This allows your customers to get help from a live agent in real time, so that they can resolve their issues quickly and efficiently.
* **Mixtral 8x7B can be used to track customer satisfaction.** This information can be used to identify areas where you can improve your customer service and provide a more personalized experience.
* **Mixtral 8x7B can be used to create surveys and polls.** This feedback can be used to improve your products and services, and to identify new opportunities to grow your business.